+ Lexicon => 사전(해시 테이블) => 단어:위치, 단어:위치, ... <br>
+ Posting => 문서:빈도:다음위치, 문서:빈도:다음위치, ... <br>
+ Local Indexing => Merge(위치를 조정, linked list) <br>
+ Collection(문서집합), Lexicon(사전)

In [16]:
collection = {
    ('Document1', 'This is a sample'),
    ('Document2', 'This is another sample'),
    ('Document3', 'This is not a sample')
}

globalLexicon = dict()
globalDocument = list()
globalPosting = list()

In [17]:
for docName, docContent in collection:
    docIdx = len(globalDocument)
    globalDocument.append(docName)
    
    localPosting = dict()
    for term in docContent.lower().split():
        if term in localPosting.keys():
            localPosting[term] += 1      
        else:
            localPosting[term] = 1
            
    ############################################# Local   
    
    for term, freq in localPosting.items():
        if term in globalLexicon.keys():
            # Lexicon term ? => Posting 기록, 위치를 업데이트
            termIdx = list(globalLexicon.keys()).index(term)
            postingIdx = len(globalPosting)
            globalPosting.append((docIdx, freq, globalLexicon[term]))
            globalLexicon[term] = postingIdx
        else:
            # Posting 기록, 위치도 기록
            termIdx = len(globalLexicon.keys())
            postingIdx = len(globalPosting)       
            globalPosting.append((docIdx, freq, -1))
            globalLexicon[term] = postingIdx
            
globalLexicon, globalPosting[8], globalPosting[4], globalPosting[0]

({'this': 9, 'is': 10, 'not': 2, 'a': 11, 'sample': 12, 'another': 7},
 (1, 1, 4),
 (0, 1, -1),
 (0, 1, -1))

In [18]:
for term, postingIdx in globalLexicon.items(): # TDM.keys()
    print(term)
    
    while True:
        if postingIdx == -1:
            break
         
        data = globalPosting[postingIdx]
        print(' {0}-{1}, {2}, Next={3}'.format(data[0], 
                                               globalDocument[data[0]],
                                               data[1],
                                               data[2]))
        postingIdx = data[2]

this
 2-Document1, 1, Next=5
 1-Document2, 1, Next=0
 0-Document3, 1, Next=-1
is
 2-Document1, 1, Next=6
 1-Document2, 1, Next=1
 0-Document3, 1, Next=-1
not
 0-Document3, 1, Next=-1
a
 2-Document1, 1, Next=3
 0-Document3, 1, Next=-1
sample
 2-Document1, 1, Next=8
 1-Document2, 1, Next=4
 0-Document3, 1, Next=-1
another
 1-Document2, 1, Next=-1


## TF

In [19]:
# TF
# 특정 문서 내 특정 단어의 빈도 = f(t,d) - OK
# 특정 문서 내 다른 단어의 빈도 => sum, max - DTM(sum, max)
from math import log10

def logTF(freq):
    return log10(1+freq)

def totalTF(freq, totalFreq):
    return freq/totalFreq

def doubleTF(freq, maxFreq, alpha=0.5):
    return alpha+(1-alpha)*(freq/maxFreq)

## IDF

In [20]:
# IDF
# 전체 문서의 갯수 => N
# 특정 단어가 나타난 문서의 갯수 => dt(t)
def idf(df, N):
    return log10(N/df)

def smoothingIdf(df, N):
    return log10(N/(1+df))

def probabilityIdf(df, N):
    return log10((N-df+1)/df)

In [21]:
N = len(globalDocument)

for term, postingIdx in globalLexicon.items():
    df = 0    
    while True:
        if postingIdx == -1:
            break
        df += 1
        data = globalPosting[postingIdx]
        
        postingIdx = data[2]
    print('{0} - DF:{1}, N:{2}'.format(term, df, N))
    print('    IDF1:{0}'.format(idf(df, N)))
    print('    IDF2:{0}'.format(smoothingIdf(df, N)))
    print('    IDF3:{0}'.format(probabilityIdf(df, N)))
    print()

this - DF:3, N:3
    IDF1:0.0
    IDF2:-0.12493873660829995
    IDF3:-0.47712125471966244

is - DF:3, N:3
    IDF1:0.0
    IDF2:-0.12493873660829995
    IDF3:-0.47712125471966244

not - DF:1, N:3
    IDF1:0.47712125471966244
    IDF2:0.17609125905568124
    IDF3:0.47712125471966244

a - DF:2, N:3
    IDF1:0.17609125905568124
    IDF2:0.0
    IDF3:0.0

sample - DF:3, N:3
    IDF1:0.0
    IDF2:-0.12493873660829995
    IDF3:-0.47712125471966244

another - DF:1, N:3
    IDF1:0.47712125471966244
    IDF2:0.17609125905568124
    IDF3:0.47712125471966244



In [22]:
collection = {
    ('Document1', 'This is a a a a a a a a sample'),
    ('Document2', 'This is another sample'),
    ('Document3', 'This is not a sample')
}

globalLexicon = dict()
globalDocument = list()
globalPosting = list()
globalMaxTF = dict()
globalTotalTF = dict()

In [23]:
for docName, docContent in collection:
    docIdx = len(globalDocument)
    globalDocument.append(docName)
    
    localPosting = dict()
    for term in docContent.lower().split():
        if term in localPosting.keys():
            localPosting[term] += 1      
        else:
            localPosting[term] = 1
            
    globalMaxTF[docIdx] = max(localPosting.values())
    globalTotalTF[docIdx] = sum(localPosting.values())
    
    ############################################# Local   
    
    for term, freq in localPosting.items():
        if term in globalLexicon.keys():
            # Lexicon term ? => Posting 기록, 위치를 업데이트
            termIdx = list(globalLexicon.keys()).index(term)
            postingIdx = len(globalPosting)
            globalPosting.append((docIdx, freq, globalLexicon[term]))
            globalLexicon[term] = postingIdx
        else:
            # Posting 기록, 위치도 기록
            termIdx = len(globalLexicon.keys())
            postingIdx = len(globalPosting)       
            globalPosting.append((docIdx, freq, -1))
            globalLexicon[term] = postingIdx

In [24]:
N = len(globalDocument)

for term, postingIdx in globalLexicon.items():
    df = 0    
    while True:
        if postingIdx == -1:
            break
        df += 1
        data = globalPosting[postingIdx]
        
        print('{0} - DocIdx:{1}, TF:{2}, Max{3}, Total:{4}'.format(
            term, data[0], data[1], globalMaxTF[data[0]], globalTotalTF[data[0]]))
        
        print('   TF1:{0}'.format(logTF(data[1])))
        print('   TF2:{0}'.format(totalTF(data[1], globalTotalTF[data[0]])))
        print('   TF3:{0}'.format(doubleTF(data[1], globalMaxTF[data[0]])))
        
        postingIdx = data[2]
        
    print('{0} - DF:{1}, N:{2}'.format(term, df, N))
    print('    IDF1:{0}'.format(idf(df, N)))
    print('    IDF2:{0}'.format(smoothingIdf(df, N)))
    print('    IDF3:{0}'.format(probabilityIdf(df, N)))
    print()

this - DocIdx:2, TF:1, Max1, Total:4
   TF1:0.3010299956639812
   TF2:0.25
   TF3:1.0
this - DocIdx:1, TF:1, Max8, Total:11
   TF1:0.3010299956639812
   TF2:0.09090909090909091
   TF3:0.5625
this - DocIdx:0, TF:1, Max1, Total:5
   TF1:0.3010299956639812
   TF2:0.2
   TF3:1.0
this - DF:3, N:3
    IDF1:0.0
    IDF2:-0.12493873660829995
    IDF3:-0.47712125471966244

is - DocIdx:2, TF:1, Max1, Total:4
   TF1:0.3010299956639812
   TF2:0.25
   TF3:1.0
is - DocIdx:1, TF:1, Max8, Total:11
   TF1:0.3010299956639812
   TF2:0.09090909090909091
   TF3:0.5625
is - DocIdx:0, TF:1, Max1, Total:5
   TF1:0.3010299956639812
   TF2:0.2
   TF3:1.0
is - DF:3, N:3
    IDF1:0.0
    IDF2:-0.12493873660829995
    IDF3:-0.47712125471966244

not - DocIdx:0, TF:1, Max1, Total:5
   TF1:0.3010299956639812
   TF2:0.2
   TF3:1.0
not - DF:1, N:3
    IDF1:0.47712125471966244
    IDF2:0.17609125905568124
    IDF3:0.47712125471966244

a - DocIdx:1, TF:8, Max8, Total:11
   TF1:0.9542425094393249
   TF2:0.7272727272727273

In [25]:
N = len(globalDocument)

for term, postingIdx in globalLexicon.items():
    old = postingIdx
    df = 0    
    while True:
        if postingIdx == -1:
            break
        df += 1
        postingIdx = globalPosting[postingIdx][2]
        
    
    postingIdx = old
    idf1 = idf(df, N)
    idf2 = smoothingIdf(df, N)
    idf3 = probabilityIdf(df, N)
    
    while True:
        if postingIdx == -1:
            break

        data = globalPosting[postingIdx]
        postingIdx = data[2]
        tf = doubleTF(data[1], globalMaxTF[data[0]])
        print('{0}-{1}'.format(term, globalDocument[data[0]]))
        print(' => {0} * {1} = {2}'.format(tf, idf1, tf*idf1))
        print(' => {0} * {1} = {2}'.format(tf, idf2, tf*idf2))
        print(' => {0} * {1} = {2}'.format(tf, idf3, tf*idf3))
    print()

this-Document2
 => 1.0 * 0.0 = 0.0
 => 1.0 * -0.12493873660829995 = -0.12493873660829995
 => 1.0 * -0.47712125471966244 = -0.47712125471966244
this-Document1
 => 0.5625 * 0.0 = 0.0
 => 0.5625 * -0.12493873660829995 = -0.07027803934216872
 => 0.5625 * -0.47712125471966244 = -0.2683807057798101
this-Document3
 => 1.0 * 0.0 = 0.0
 => 1.0 * -0.12493873660829995 = -0.12493873660829995
 => 1.0 * -0.47712125471966244 = -0.47712125471966244

is-Document2
 => 1.0 * 0.0 = 0.0
 => 1.0 * -0.12493873660829995 = -0.12493873660829995
 => 1.0 * -0.47712125471966244 = -0.47712125471966244
is-Document1
 => 0.5625 * 0.0 = 0.0
 => 0.5625 * -0.12493873660829995 = -0.07027803934216872
 => 0.5625 * -0.47712125471966244 = -0.2683807057798101
is-Document3
 => 1.0 * 0.0 = 0.0
 => 1.0 * -0.12493873660829995 = -0.12493873660829995
 => 1.0 * -0.47712125471966244 = -0.47712125471966244

not-Document3
 => 1.0 * 0.47712125471966244 = 0.47712125471966244
 => 1.0 * 0.17609125905568124 = 0.17609125905568124
 => 1.0 * 0.